# **Large-Scale Kinship Recognition Data Challenge: Kinship Verification STARTER NOTEBOOK**

We provide framework code to get you started on the competition. The notebook is broken up into three main sections. 
1. Data Loading & Visualizing
2. Data Generator & Model Building
3. Training & Testing Model

We have done the majority of the heavy lifting by making the data easily and readily accessible through Google Drive. Furthermore, we have made the task easier by creating a dataloader and fully trained end-to-end model that predicts a binary label (0 or 1) denoting whether two faces share a kinship relation. 

Mount to Google Drive

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


Install Libraries

In [4]:
%%capture
!pip install keras_vggface
!pip install keras_applications

In [5]:
from collections import defaultdict
from glob import glob
from random import choice, sample

import tensorflow as tf
import keras
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Dense, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace

[link text](https://)train_relationships.csv contains pairs of image paths which are positive samples (related to each other).

train-faces contains the images for training itself.

In [6]:
# Modify paths as per your method of saving them
train_file_path = "/gdrive/MyDrive/Kinship Recognition Starter/train_ds.csv"
#!ls /gdrive/MyDrive/
train_folders_path = "/gdrive/MyDrive/Kinship Recognition Starter/train/train-faces/"
# All images belonging to families F09** will be used to create the validation set while training the model
# For final submission, you can add these to the training data as well
val_famillies = "F09"

all_images = glob(train_folders_path + "*/*/*.jpg") #all images

train_images = [x for x in all_images if val_famillies not in x] #all images except for F09*
val_images = [x for x in all_images if val_famillies in x] #all images that are F09*

ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in all_images] #family/member/ for all images

train_person_to_images_map = defaultdict(list)
for x in train_images:
    train_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x) #add a training person to map

val_person_to_images_map = defaultdict(list)
for x in val_images:
    val_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x) #add a validation person to map

relationships = pd.read_csv(train_file_path)
relationships = list(zip(relationships.p1.values, relationships.p2.values, relationships.relationship.values))
relationships = [(x[0],x[1],x[2]) for x in relationships if x[0][:10] in ppl and x[1][:10] in ppl]

train = [x for x in relationships if val_famillies not in x[0]]
val = [x for x in relationships if val_famillies in x[0]]

from keras.preprocessing import image
def read_img(path):
    img = image.load_img(path, target_size=(224, 224))
    img = np.array(img).astype(np.float)
    return preprocess_input(img, version=2)

def gen(list_tuples, person_to_images_map, batch_size=16):
    ppl = list(person_to_images_map.keys())
    while True:
        batch_tuples = sample(list_tuples, batch_size)
        
        # All the samples are taken from train_ds.csv, labels are in the labels column
        labels = []
        for tup in batch_tuples:
          labels.append(tup[2])

        X1 = [x[0] for x in batch_tuples]
        X1 = np.array([read_img(train_folders_path + x) for x in X1])

        X2 = [x[1] for x in batch_tuples]
        X2 = np.array([read_img(train_folders_path + x) for x in X2])

        yield [X1, X2], np.array(labels)

Here is an ensemble model built with two resnet-50 architectures, pre-trained, with which we can apply transfer leraning on. This model achieves the baseline and the goal is to expand on this work. There have been papers exploring different architectures as well as introducing BatchNormalization among many other techniques to improve how well the model recognizes kinship between two faces.

In [19]:
from tensorflow.keras.layers import Conv2D, Flatten, Lambda, Add
from keras import backend as K

def baseline_model():
    input_1 = Input(shape=(224, 224, 3))
    input_2 = Input(shape=(224, 224, 3))

    base_model = VGGFace(model='resnet50', include_top=False)

    for x in base_model.layers[:-3]:
        x.trainable = True

    #INPUT LAYER
    x1 = base_model(input_1) #reshaping input of model to that of image shapes
    x2 = base_model(input_2) #requries two resnet archs

    #RESNET OUTPUT LAYER CONCAT
    x1 = Concatenate(axis=-1)([GlobalMaxPool2D()(x1), GlobalAvgPool2D()(x1)]) #not too sure
    x2 = Concatenate(axis=-1)([GlobalMaxPool2D()(x2), GlobalAvgPool2D()(x2)])

    #lambda_1 = Lambda(lambda tensor  : K.square(tensor))(x1)
    #lambda_2 = Lambda(lambda tensor  : K.square(tensor))(x2)

    #added = Add()([x1, x2])
    #subtracted = Subtract()([x1, x2])
    #subtracted2 = Subtract()([x2, x1])
    #multiplied = Multiply()([x1,x2])
    #sq_sum = Add()([lambda_1,lambda_2])
    #sqrt = Lambda(lambda tensor  : K.sign(tensor)*K.sqrt(K.abs(tensor)+1e-9))(multiplied) #squre_root of sqrt_vgg1

    x3 = Subtract()([x1, x2]) #substract x1 and x2
    x3 = Multiply()([x3, x3]) #then square it

    x = Multiply()([x1, x2]) #multiply x1 and x2
    x = Concatenate(axis=-1)([x, x3]) #concatenate (multiply x1 and x2) with (substract x1 and x2, then square)

    #concatenated = Concatenate(axis=-1)([Flatten()(added),
    #                                     Flatten()(subtracted),
    #                                     Flatten()(subtracted2), 
    #                                     Flatten()(multiplied),
    #                                     Flatten()(sq_sum), 
    #                                     Flatten()(sqrt)])
    
    concatenated = Dense(500, activation="relu")(x)
    concatenated = Dropout(0.1)(concatenated)
    concatenated = Dense(100, activation="relu")(concatenated)
    concatenated = Dropout(0.1)(concatenated)
    concatenated = Dense(25, activation="relu")(concatenated)
    concatenated = Dropout(0.1)(concatenated)
    out = Dense(1, activation="sigmoid")(concatenated)
    #x = Dense(100, activation="relu")(x) #output 100-dimension
    #x = Dropout(0.05)(x) #adding regularization
    #out = Dense(1, activation="sigmoid")(x) #output 1 (classification)

    model = Model([input_1, input_2], out)
    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.00001))

    model.summary()

    return model

Save the best model to your drive after each training epoch so that you can come back to it. ReduceLROnPlateau reduces the learning rate when a metric has stopped improving, in this case the validation accuracy. 

In [20]:
file_path = "/gdrive/MyDrive/vgg_face.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='max')
reduce_on_plateau = ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.1, patience=20, verbose=1)
callbacks_list = [checkpoint, reduce_on_plateau]
model = baseline_model()

The following Variables were used a Lambda layer's call (tf.nn.convolution_318), but
are not present in its tracked objects:
  <tf.Variable 'conv1/7x7_s2/kernel:0' shape=(7, 7, 3, 64) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.compat.v1.nn.fused_batch_norm_318), but
are not present in its tracked objects:
  <tf.Variable 'conv1/7x7_s2/bn/gamma:0' shape=(64,) dtype=float32>
  <tf.Variable 'conv1/7x7_s2/bn/beta:0' shape=(64,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.convolution_319), but
are not present in its tracked objects:
  <tf.V

In [21]:
model.fit(gen(train, train_person_to_images_map, batch_size=16), use_multiprocessing=False,
                validation_data=gen(val, val_person_to_images_map, batch_size=16), epochs=30, verbose=1,
                workers=1, callbacks=callbacks_list, steps_per_epoch=100, validation_steps=50)

Epoch 1/30
100/100 [==============================] - 24s 202ms/step - loss: 4.0165 - acc: 0.5788 - val_loss: 2.0549 - val_acc: 0.6275

Epoch 00001: val_acc improved from -inf to 0.62750, saving model to /gdrive/MyDrive/vgg_face.h5
Epoch 2/30
100/100 [==============================] - 19s 192ms/step - loss: 2.3232 - acc: 0.6637 - val_loss: 1.5751 - val_acc: 0.6325

Epoch 00002: val_acc improved from 0.62750 to 0.63250, saving model to /gdrive/MyDrive/vgg_face.h5
Epoch 3/30
100/100 [==============================] - 19s 188ms/step - loss: 1.5657 - acc: 0.7156 - val_loss: 1.2574 - val_acc: 0.6550

Epoch 00003: val_acc improved from 0.63250 to 0.65500, saving model to /gdrive/MyDrive/vgg_face.h5
Epoch 4/30
100/100 [==============================] - 18s 185ms/step - loss: 1.2119 - acc: 0.7531 - val_loss: 1.2281 - val_acc: 0.6538

Epoch 00004: val_acc did not improve from 0.65500
Epoch 5/30
100/100 [==============================] - 19s 195ms/step - loss: 0.9315 - acc: 0.7844 - val_loss: 1.

In [23]:
# Modify paths as per your need
test_path = "/gdrive/MyDrive/Kinship Recognition Starter/test/"

#model = baseline_model()
#model.load_weights("/gdrive/MyDrive/vgg_face.h5")

submission = pd.read_csv('/gdrive/MyDrive/Kinship Recognition Starter/test_ds.csv')
predictions = []

for i in range(0, len(submission.p1.values), 32):
    if i%100 == 0:
      print(i)
    X1 = submission.p1.values[i:i+32]
    X1 = np.array([read_img(test_path + x) for x in X1])

    X2 = submission.p2.values[i:i+32]
    X2 = np.array([read_img(test_path + x) for x in X2])

    pred = model.predict([X1, X2]).ravel().tolist()
    predictions += pred

0
800
1600
2400


The final predictions will need to be rounded: EG 0.01 rounded to 0 and 0.78 rounded to 1. The simple .round() function is sufficient as below.

In [25]:
d = {'index': np.arange(0, 3000, 1), 'label':predictions}
submissionfile = pd.DataFrame(data=d)
submissionfile = submissionfile.round()
submissionfile.astype("int64").to_csv("/gdrive/MyDrive/tojo.csv", index=False)

At this point, download the CSV and submit it on Kaggle to score your predictions.
